# Продвинутое машинное обучение: ДЗ 2
## *Латыпов Ильяс, группа DS-22*
### Часть 1.

Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
   - взять в тренировочный набор турниры с dateStart из 2019 года; 
   - в тестовый — турниры с dateStart из 2020 года.


In [85]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression, LinearRegression

import zipfile
import pickle

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()


# from collections import defaultdict
# from scipy.stats import spearmanr, kendalltau

# Загрузка и обработка данных

In [2]:
myzip = zipfile.ZipFile("D:\\ID\\Course\\MADE\\2.Adv_ML\\HW\\chgk.zip")
print("Содержание zip файла:")
for file_name in myzip.namelist():
    print(file_name)

Содержание zip файла:
players.pkl
results.pkl
tournaments.pkl


### Загрузка tournaments.pkl

In [3]:
data_tournament = pd.DataFrame(pickle.load(myzip.open('tournaments.pkl', 'r'))).T
data_tournament["year"] = data_tournament['dateStart'].apply(lambda x: int(x[:4]) if x is not None else None)
data_tournament['sum_questionQty'] = data_tournament['questionQty'].apply(lambda x: sum(x.values()) if x is not None else 0)
train_tournament = data_tournament[data_tournament['year']==2019]
test_tournament = data_tournament[data_tournament['year']==2020]
train_tournament = train_tournament[["id", "year", "sum_questionQty"]]
test_tournament = test_tournament[["id", "year", "sum_questionQty"]]
print("Количество соревнований в 2019 в tournaments.pkl:", train_tournament.shape)
print("Количество соревнований в 2020 в tournaments.pkl:", test_tournament.shape)

Количество соревнований в 2019 в tournaments.pkl: (687, 3)
Количество соревнований в 2020 в tournaments.pkl: (418, 3)


### Загрузка players.pkl

In [4]:
data_players = pd.DataFrame(pickle.load(myzip.open('players.pkl', 'r'))).T
print("Общее количество участников в players.pkl:", data_players.shape)
display(data_players.sample(2))

Общее количество участников в players.pkl: (204063, 4)


,id,name,patronymic,surname
217116,217116,Владимир,Александрович,Варивода
152643,152643,Ульяна,Юрьевна,Кобылец


### Загрузка results.pkl

In [25]:
%%time
results = pickle.load(myzip.open('results.pkl', 'r'))
tournament_test_list = test_tournament["id"].to_list() 
tournament_train_list = train_tournament["id"].to_list()

data_result = pd.DataFrame([])
for tour in (tournament_train_list + tournament_test_list):
    if tour not in results.keys():
        print(f"Не найдено соревнование № {tour} в results.pkl")
        break
    cur_tour_df = pd.DataFrame(results[tour])
    cur_tour_df["tournament"] = tour
    if tour in tournament_train_list:
        cur_tour_df["year"] = 2019
    else:
        cur_tour_df["year"] = 2020
    data_result = pd.concat([data_result, cur_tour_df])
    
data_result["teamMembers_empty"] = data_result["teamMembers"].apply(lambda x: not x) 
data_result = data_result[(~data_result["mask"].isna()) & (~data_result["teamMembers_empty"])]
data_result.drop(columns=["teamMembers_empty"], inplace=True)

data_result["players_id"] = data_result["teamMembers"].apply(lambda x: list(map(lambda y: y['player']['id'], x)))
data_result["players_number"] = data_result["teamMembers"].apply(lambda x: len(list(map(lambda y: y['player']['id'], x))))
data_result["players_name"] = data_result["teamMembers"].apply(lambda x: list(map(lambda y: y['player']['surname'] + " " +
                                                               y['player']['name'] + " " + y['player']['patronymic'], x)))

data_result["team_id"] = data_result["team"].apply(lambda x: x['id'])
data_result = data_result[["tournament", "year", "team_id", "mask", "questionsTotal", 
                           "players_id", "players_name", "players_number"]]

# data_result = data_result[~data_result['mask'].apply(lambda x: "X" in x)]
data_result['mask'] = data_result['mask'].apply(lambda x: x.replace("?","0").replace("-","0").replace("X",""))
data_result["mask_length"] = data_result["mask"].apply(len)

temp = pd.DataFrame(data_result.groupby(by = "tournament")["mask_length"].max())
temp.rename(columns = {"mask_length" : "tour_mask_length"}, inplace=True)
data_result = pd.merge(data_result, temp, on='tournament', how="left")
data_result = data_result[data_result["mask_length"] == data_result["tour_mask_length"]]
data_result.drop(columns=["tour_mask_length"], inplace=True)

# temp = data_tournament[(data_tournament['year']==2019) | (data_tournament['year']==2020)][["id", "sum_questionQty"]]
# temp.rename(columns = {"id" : "tournament"}, inplace=True)
# data_result = pd.merge(data_result, temp, on='tournament', how="left")
# data_result = data_result[data_result["mask_length"]==data_result["sum_questionQty"]]

data_result_ext = data_result.loc[data_result.index.repeat(data_result.players_number)]
data_result_ext = data_result_ext.reset_index()
data_result_ext.drop(columns = ["index"], inplace=True)
data_result_ext = data_result_ext.reset_index()
data_result_ext["list_ind"] = data_result_ext["index"] % data_result_ext["players_number"]
data_result_ext["player_id"] = data_result_ext.apply(lambda x: x["players_id"][x["list_ind"]], axis=1)
data_result_ext["player_name"] = data_result_ext.apply(lambda x: x["players_name"][x["list_ind"]], axis=1)
data_result_ext.drop(columns=["players_id","players_name", "players_number", "list_ind", "index"], inplace=True)
data_result_ext.rename(columns = {"tournament" : "tournament_id"}, inplace=True)

train_result = data_result_ext[data_result_ext["year"] == 2019]
test_result = data_result_ext[data_result_ext["year"] == 2020]
test_result = test_result[test_result["player_id"].isin(set(train_result.player_id.to_list()))]
train_result.to_pickle('result_train.pkl')
test_result.to_pickle('result_test.pkl')

print("Сумма участников во всех турнирах 2019г", train_result.shape)
print("Сумма участников во всех турнирах 2020г:", test_result.shape)
display(train_result.head(2))

Сумма участников во всех турнирах 2019г (414774, 8)
Сумма участников во всех турнирах 2020г: (99757, 8)


,tournament_id,year,team_id,mask,questionsTotal,mask_length,player_id,player_name
0,4772,2019,45556,111111111011111110111111111100010010,28.0,36,6212,Выменец Юрий Яковлевич
1,4772,2019,45556,111111111011111110111111111100010010,28.0,36,18332,Либер Александр Витальевич


Wall time: 1min 40s


In [26]:
print("Выборка соревнований для обучения:", data_result[data_result["year"] == 2019].tournament.nunique())
print("Выборка соревнований для теста:", data_result[data_result["year"] == 2020].tournament.nunique())

Выборка соревнований для обучения: 675
Выборка соревнований для теста: 173


In [176]:
train_result = pd.read_pickle('result_train.pkl')
test_result = pd.read_pickle('result_test.pkl')
train_result.shape, test_result.shape

((414774, 8), (99757, 8))

In [177]:
players_train_ids = set(train_result.player_id.to_list())
players_test_ids = set(test_result.player_id.to_list())
print("Количество участников в 2019г: ", len(players_train_ids))
print("Количество участников в 2020г: ", len(players_test_ids))

Количество участников в 2019г:  57424
Количество участников в 2020г:  22946


### Часть 2.

Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
  - повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
  - в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
  - для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


In [178]:
%%time
enum_players_ids = {k: i for i, k in enumerate(players_train_ids)}
enum_train_tournament_ids = {k: i for i, k in enumerate(set(train_result["tournament_id"].to_list()))}
enum_test_tournament_ids = {k: i for i, k in enumerate(set(test_result["tournament_id"].to_list()))}
train_result['player_dop_id'] = train_result['player_id'].map(enum_players_ids)
train_result['tournament_dop_id'] = train_result['tournament_id'].map(enum_train_tournament_ids)                                                                   
test_result['player_dop_id'] = test_result['player_id'].map(enum_players_ids)
test_result['tournament_dop_id'] = test_result['tournament_id'].map(enum_test_tournament_ids)

temp = train_result.groupby('tournament_dop_id')['mask_length'].max().cumsum()
question_numbers = temp.iloc[-1]
question_start_ids = dict(temp - temp[0])
X = list()
y = list()
for ind in range(train_result.shape[0]):
    question_start_id = question_start_ids[train_result.iloc[ind]["tournament_dop_id"]]
    player_id = train_result.iloc[ind]['player_dop_id']
    mask = train_result.iloc[ind]['mask']
    for i, question_result in enumerate(mask):
        X.append([question_start_id + i, player_id])                
        if question_result == '0':
            y.append(0)
        else:
            y.append(1)
        
X = np.array(X)
y = np.array(y)

def convert_to_csr_matrix(X, question_numbers):
    x_rows = np.repeat(np.arange(X.shape[0]), 2)
    x_new = X.copy()
    x_new[:,1] += question_numbers + 1
    csr_x = csr_matrix((np.ones(X.shape[0] * 2, dtype=np.uint8), (x_rows, x_new.ravel())), dtype=np.uint8)
    return csr_x

X_csr = convert_to_csr_matrix(X, question_numbers)
X_csr.shape, y.shape

Wall time: 2min 51s


((17753544, 90728), (17753544,))

In [179]:
%%time
model = LogisticRegression(max_iter=300, solver="lbfgs", fit_intercept=False)
model.fit(X_csr, y)
print("Mean accuracy:", model.score(X_csr, y))

Mean accuracy: 0.7609300430381675
Wall time: 2min 15s


In [209]:
rating = {ind:weight for ind, weight in enumerate(model.coef_[0, question_numbers + 1:])}
ratings = train_result[['player_dop_id', 'player_name']].drop_duplicates()
ratings['rating'] = ratings['player_dop_id'].apply(lambda x: rating[x])
ratings = ratings.sort_values(by='rating', ascending=False)
ratings = ratings.reset_index(drop=True)
ratings.head(15)

,player_dop_id,player_name,rating
0,9646,Руссо Максим Михайлович,3.360885
1,1452,Брутер Александра Владимировна,3.247109
2,10176,Семушин Иван Николаевич,3.221678
3,9831,Савченков Михаил Владимирович,3.115753
4,10660,Сорожкин Артём Сергеевич,3.046544
5,10700,Спешков Сергей Леонидович,3.043067
6,7120,Мереминский Станислав Григорьевич,2.908851
7,51130,Саксонов Антон Владимирович,2.812316
8,9154,Прокофьева Ирина Сергеевна,2.788269
9,6180,Левандовский Михаил Ильич,2.775282
